NOTA: Variables filtradas por cada filtro
  1) Filtro #1 -> Variable: CVE_CLPV (rango 1000 a 2000, inclusivo)
  2) Filtro #2 -> Variable: CVE_VEND (todas EXCEPTO claves '5' y '4')
  3) Filtro #3 -> Variable: FECHA_ENT (solo fecha 28/02/2022)
  4) Filtro #4 -> Variables: CAN_TOT (< 5951.7)  OR  STATUS == 'E'
  5) Filtro #5 -> Columnas seleccionadas: CVE_DOC, FECHA_ENT, FECHA_VEN, CAN_TOT
  6) Filtro #6 -> Filas 7001 a 7099 (basado en numeración humana 1..N)
  7) Filtro #7 -> Variable: CVE_VEND in {1, 2}; se establece índice CVE_VEND
                  y se conserva columna FECHAELAB (si existe)

In [150]:
#Importar librerías
import os
import sys
import pandas as pd

In [151]:
#Configuración de rutas
INPUT_PATH = r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Extraccion_de_Datos\Datos_Facturación.csv"
OUTPUT_DIR = r"C:\Users\navae\OneDrive\Escritorio\Tec\Semestre 5\Plataformas de analítica de negocios para organizaciones\Clonación Github\Extraccion_de_Datos"


In [152]:
# Leer CSV
df = pd.read_csv(INPUT_PATH)

In [153]:
# Vistazo rápido a los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10895 entries, 0 to 10894
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     10895 non-null  int64  
 1   CVE_DOC        10890 non-null  object 
 2   CVE_CLPV       10895 non-null  float64
 3   STATUS         10886 non-null  object 
 4   CVE_VEND       10847 non-null  float64
 5   FECHAELAB      10893 non-null  object 
 6   FECHA_DOC      10894 non-null  object 
 7   FECHA_ENT      10893 non-null  object 
 8   FECHA_VEN      10895 non-null  object 
 9   FECHA_CANCELA  358 non-null    object 
 10  CAN_TOT        10889 non-null  float64
 11  DES_TOT        10894 non-null  float64
 12  DES_FIN        10892 non-null  float64
dtypes: float64(5), int64(1), object(7)
memory usage: 1.1+ MB


In [154]:
# Filtro 1: CVE_CLPV en [1000, 2000]
if "CVE_CLPV" in df.columns:
    f1 = df[(df["CVE_CLPV"] >= 1000) & (df["CVE_CLPV"] <= 2000)]
else:
    f1 = pd.DataFrame()

#Guardar
f1_path = os.path.join(OUTPUT_DIR, "Filtro1_CVE_CLPV_1000_2000.csv")
f1.to_csv(f1_path, index=False, encoding="utf-8-sig")


In [155]:
# Filtro 2: CVE_VEND != {"5","4"}
if "CVE_VEND" in df.columns:
    # Convertir a string 
    vend_str = df["CVE_VEND"].astype(str)
    f2 = df[~vend_str.isin(["5.0", "4.0"])]
else:
    f2 = pd.DataFrame()
  
#Guardar
f2_path = os.path.join(OUTPUT_DIR, "Filtro2_CVE_VEND_excluir_5_4.csv")
f2.to_csv(f2_path, index=False, encoding="utf-8-sig")

In [156]:
# Filtro 3: FECHA_ENT == 28/02/2022 (rango del día)
if "FECHA_ENT" in df.columns:
    fe_str = df["FECHA_ENT"].astype(str).str.strip()
    fe = pd.to_datetime(fe_str, errors="coerce", infer_datetime_format=True)

    start = pd.Timestamp("2022-02-28 00:00:00")
    end   = pd.Timestamp("2022-03-01 00:00:00")
    mask = (fe >= start) & (fe < end)

    f3 = df.loc[mask].copy()
else:
    f3 = pd.DataFrame()

#Guardar
f3_path = os.path.join(OUTPUT_DIR, "Filtro3_FECHA_ENT_2022-02-28.csv")
f3.to_csv(f3_path, index=False, encoding="utf-8-sig")

C:\Users\navae\AppData\Local\Temp\ipykernel_21460\1036824748.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  fe = pd.to_datetime(fe_str, errors="coerce", infer_datetime_format=True)


In [157]:
# Filtro 4: CAN_TOT < 5951.7  OR  STATUS == "E" 
cond_left = pd.Series(False, index=df.index)
if "CAN_TOT" in df.columns:
    cond_left = df["CAN_TOT"] < 5951.7

cond_right = pd.Series(False, index=df.index)
if "STATUS" in df.columns:
    cond_right = df["STATUS"].astype(str) == "E"

f4 = df[cond_left | cond_right]

#Guardar
f4_path = os.path.join(OUTPUT_DIR, "Filtro4_CAN_TOT_menor_5951_7_o_STATUS_E.csv")
f4.to_csv(f4_path, index=False, encoding="utf-8-sig")

In [158]:
# Filtro 5: Solo columnas
cols = ["CVE_DOC", "FECHA_ENT", "FECHA_VEN", "CAN_TOT"]
existing_cols = [c for c in cols if c in df.columns]
f5 = df[existing_cols].copy()

#Guardar
f5_path = os.path.join(OUTPUT_DIR, "Filtro5_columnas_CVE_DOC_FECHA_ENT_FECHA_VEN_CAN_TOT.csv")
f5.to_csv(f5_path, index=False, encoding="utf-8-sig")

In [159]:
# Filtro 6: Solo filas 7001-7099
f6 = df.iloc[7001:7100].copy()

# Guardar
f6_path = os.path.join(OUTPUT_DIR, "Filtro6_filas_7001_7099.csv")
f6.to_csv(f6_path, index=False, encoding="utf-8-sig")

In [160]:
# Filtro 7: Index = CVE_VEND (valores 1,2) con columna FECHAELAB
if "CVE_VEND" in df.columns:
    f7 = df[df["CVE_VEND"].isin([1, 2])].set_index("CVE_VEND")[["FECHAELAB"]].copy()
else:
    f7 = pd.DataFrame()

# Guardar
f7_path = os.path.join(OUTPUT_DIR, "Filtro7_index_CVE_VEND_vals_1_2_col_FECHAELAB.csv")
f7.to_csv(f7_path, encoding="utf-8-sig")

